In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import seaborn as sns
import tensorflow as tf
import sklearn 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from tensorflow import keras
from keras import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU, BatchNormalization
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import f1_score, precision_score, recall_score

go_back = '../data/'
go_back + 

In [ ]:
print("Pandas version: ", pd.__version__)
print("Numpy version: ", np.__version__)
print("Matplotlib version: ", matplotlib.__version__)
print("Seaborn version: ", sns.__version__)
print("Tensorflow version: ", tf.__version__)
print("Scikit-learn version: ", sklearn.__version__)
print("Keras version: ", keras.__version__)

# Loading in Data

In [ ]:
df = pd.read_csv(go_back + 'all_data_train.csv')
df = df.drop(['Unnamed: 0'], axis = 1)
df

# Feature Selection

In [ ]:
X = df.drop(['team', 'opponent', 'outcome', 'date',
            'score', 'opponentScore', 'total', 'secondHalfTotal'], axis = 1)
Y = df['outcome']

X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
test = pd.read_csv('all_data_test.csv')
X_test = test.drop(['Unnamed: 0', 'team', 'opponent', 'outcome', 'date',
            'score', 'opponentScore', 'total', 'secondHalfTotal'], axis = 1)
y_test = test['outcome']

From prior analysis, reducing the number of features does not improve performance and actually reduces our expected profits.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

standardized_x = StandardScaler().fit_transform(X)
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(standardized_x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

In [ ]:
principalDf['target'] = Y

fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)

targets = [0, 1]
colors = ['r', 'b']
for target, color in zip(targets,colors):
    indicesToKeep = principalDf['target'] == target
    ax.scatter(principalDf.loc[indicesToKeep, 'principal component 1']
               , principalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()

There is not much separability between the two classes based on principal components alone.

# Modeling

In [ ]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression(fit_intercept= True)
lr_model.fit(X_train, y_train)
lr_train_preds = lr_model.predict(X_train)
lr_val_preds = lr_model.predict(X_val)

train_accuracy = np.mean(lr_train_preds == y_train)
val_accuracy = np.mean(lr_val_preds == y_val)
print(train_accuracy)
print(val_accuracy)

lr_test_preds = lr_model.predict(X_test)
test_accuracy = np.mean(lr_test_preds == y_test)
print(test_accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(lr_test_preds, y_test)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=lr_model.classes_)
disp.plot()

In [ ]:
print(f1_score(y_train, lr_train_preds))
print(f1_score(y_val, lr_val_preds))
print(f1_score(y_test, lr_test_preds))
print(precision_score(y_test, lr_test_preds))
print(recall_score(y_test, lr_test_preds))

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train)
train_preds = y_pred.predict(X_train)
val_preds = y_pred.predict(X_val)
train_accuracy = np.mean(train_preds == y_train)
val_accuracy = np.mean(val_preds == y_val)
print(train_accuracy)
print(val_accuracy)

gnb_test_preds = gnb.predict(X_test)
test_accuracy = np.mean(gnb_test_preds == y_test)
print(test_accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(gnb_test_preds, y_test)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=lr_model.classes_)
disp.plot()

In [ ]:
print(f1_score(y_train, train_preds))
print(f1_score(y_val, val_preds))
print(f1_score(y_test, gnb_test_preds))
print(precision_score(y_test, gnb_test_preds))
print(recall_score(y_test, gnb_test_preds))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators = 100,
                                max_depth = 10,
                                min_samples_leaf = 5,
                                random_state = 42)
rf_model.fit(X_train, y_train)

rf_train_preds = rf_model.predict(X_train)
rf_val_preds = rf_model.predict(X_val)

rf_train_accuracy = np.mean(rf_train_preds == y_train)
rf_val_accuracy = np.mean(rf_val_preds == y_val)
print(rf_train_accuracy)
print(rf_val_accuracy)

rf_test_preds = rf_model.predict(X_test)
test_accuracy = np.mean(rf_test_preds == y_test)
print(test_accuracy)

In [ ]:
cm = confusion_matrix(rf_test_preds, y_test)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=lr_model.classes_)
disp.plot()

In [ ]:
print(f1_score(y_train, rf_train_preds))
print(f1_score(y_val, rf_val_preds))
print(f1_score(y_test, rf_test_preds))
print(precision_score(y_test, rf_test_preds))
print(recall_score(y_test, rf_test_preds))

In [ ]:
from sklearn import svm
svm_model = svm.SVC()
svm_model.fit(X_train, y_train)

svm_train_preds = svm_model.predict(X_train)
svm_val_preds = svm_model.predict(X_val)

svm_train_accuracy = np.mean(svm_train_preds == y_train)
svm_val_accuracy = np.mean(svm_val_preds == y_val)
print(svm_train_accuracy)
print(svm_val_accuracy)

svm_test_preds = svm_model.predict(X_test)
test_accuracy = np.mean(svm_test_preds == y_test)
print(test_accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(svm_test_preds, y_test)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=lr_model.classes_)
disp.plot()

In [ ]:
print(f1_score(y_train, svm_train_preds))
print(f1_score(y_val, svm_val_preds))
print(f1_score(y_test, svm_test_preds))
print(precision_score(y_test, svm_test_preds))
print(recall_score(y_test, svm_test_preds))

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada_model = AdaBoostClassifier(n_estimators=50, learning_rate=0.6)
ada_model.fit(X_train, y_train)

ada_train_preds = ada_model.predict(X_train)
ada_val_preds = ada_model.predict(X_val)

ada_train_accuracy = np.mean(ada_train_preds == y_train)
ada_val_accuracy = np.mean(ada_val_preds == y_val)
print(ada_train_accuracy)
print(ada_val_accuracy)

ada_test_preds = ada_model.predict(X_test)
test_accuracy = np.mean(ada_test_preds == y_test)
print(test_accuracy)

In [ ]:
cm = confusion_matrix(ada_test_preds, y_test)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=lr_model.classes_)
disp.plot()

In [ ]:
print(f1_score(y_train, ada_train_preds))
print(f1_score(y_val, ada_val_preds))
print(f1_score(y_test, ada_test_preds))
print(precision_score(y_test, ada_test_preds))
print(recall_score(y_test, ada_test_preds))

In [ ]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

nn_model = keras.Sequential()
nn_model.add(keras.layers.Flatten())
nn_model.add(keras.layers.Dense(units = 1, use_bias = True, activation = "sigmoid"))
optimizer = tf.keras.optimizers.SGD(learning_rate=0.8)
nn_model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[metrics.binary_accuracy])

nn_model.fit(
        x=X_train,
        y=y_train,
        epochs=20,
        batch_size=500,
        verbose=0)

nn_train_preds = nn_model.predict(X_train).T[0]
nn_val_preds = nn_model.predict(X_val).T[0]

nn_train_accuracy = np.mean(nn_train_preds == y_train)
nn_val_accuracy = np.mean(nn_val_preds == y_val)
print(nn_train_accuracy)
print(nn_val_accuracy)

nn_test_preds = nn_model.predict(X_test).T[0]
test_accuracy = np.mean(nn_test_preds == y_test)
print(test_accuracy)

In [ ]:
cm = confusion_matrix(nn_test_preds, y_test)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=lr_model.classes_)
disp.plot()

In [ ]:
print(f1_score(y_train, nn_train_preds))
print(f1_score(y_val, nn_val_preds))
print(f1_score(y_test, nn_test_preds))
print(precision_score(y_test, nn_test_preds))
print(recall_score(y_test, nn_test_preds))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc


plt.figure(figsize=(8, 6))

models = [["Logistic Regression", lr_test_preds], ["Gaussian Naive Bayes", gnb_test_preds], 
['Random Forest Classifier', rf_test_preds], ["Support Vector Machine", svm_test_preds ], 
["ADA Boost Classifier", ada_test_preds],["Custom Neural Network", nn_test_preds]]

for name, preds in models:
    fpr, tpr, _ = roc_curve(y_test, preds)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=name)

plt.plot([0, 1], [0, 1], linestyle='--', color='r', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Win/Loss Prediction')
plt.legend()
plt.show()

## Deep Learning Models

In [ ]:
dl_df = pd.read_csv(go_back + 'all_train_data.csv')
dl_df = dl_df.drop(['Unnamed: 0','score', 'opponentScore', 'total', 'secondHalfTotal'], axis = 1)
dl_df


In [ ]:
# preprocessing
X = dl_df.drop(['team', 'opponent', 'outcome', 'date'], axis = 1)
y = dl_df['outcome']
X

We'll start off by turning our time series data into tensors highlighting stats of recent games. This makes the data in perfect shape for entering into our LSTM. For example, If we want to predict the sixth game of the season, the input will be all the statistics from the first five games and the output will be the result of the sixth game.

In [ ]:
#Parameters for building out our model
n_timesteps = 5 #The number of past games we want to focus on for the outcome of the current game
n_features = X.shape[1] #highlights how many features from our final tensor we are showing the model


def create_sequences(data, labels, sequence_length = n_timesteps):
    """
    Create sequences of data with corresponding labels.
    
    :param data: Scaled features for all games.
    :param labels: Outcomes for all games.
    :param sequence_length: Number of games in each sequence (e.g., 5 for games 1-5).
    :return: Sequences of features and their corresponding outcome labels.
    """
    X, y = [], []

    for i in range(len(data) - sequence_length):
        X.append(data[i:(i + sequence_length)])
        # The label for a sequence is the outcome of the game following the sequence
        y.append(labels[i + sequence_length])  # The outcome of game a based on data from games (a - sequence_length, a)
    return np.array(X), np.array(y)

def preprocess_data(df, sequence_length = n_timesteps, scale = True):

    scaler = StandardScaler()
    team_data_tensors = {}
    
    df = df.drop(['opponent', 'date'], axis=1)
    for team in df['team'].unique():
        team_df = df[df['team'] == team]
        
        # Prepare features and labels
        features = team_df.drop(columns=['team', 'outcome'])
        labels = team_df['outcome'].values  # Outcomes as labels
        
        # Scale the features if we want to
        if scale:
            features_scaled = scaler.fit_transform(features)
        else:
            features_scaled = features
        
        # Create sequences and their corresponding labels
        X_team, y_team = create_sequences(features_scaled, labels, sequence_length)
        
        # Store the 3D tensor and labels for the team
        team_data_tensors[team] = (X_team, y_team)
        
    return team_data_tensors


team_data_tensors = preprocess_data(df, n_timesteps, False)

def concatenate_dict_items(my_dict):
    first_key, first_value = next(iter(my_dict.items()))
    first_dim = first_value[0].shape[1]
    second_dim = first_value[0].shape[2]

    fin_tensor = np.empty((0, first_dim, second_dim))
    fin_label_tensor = np.empty((0, ))

    for value in my_dict.values():

        fin_tensor = np.concatenate((fin_tensor, value[0]), axis=0)
        fin_label_tensor = np.concatenate((fin_label_tensor, value[1]))

    return fin_tensor, fin_label_tensor

fin_tensor, fin_label_tensor = concatenate_dict_items(team_data_tensors) #Tensors for training

In [ ]:
#Defining test data that we will use for evaluation as well as training and validation splits
test_df = pd.read_csv('all_data_test.csv').drop(['Unnamed: 0','score', 'opponentScore', 'total', 'secondHalfTotal'], axis = 1)
team_data_tensors = preprocess_data(test_df, n_timesteps, False)
fin_test_tensor, fin_test_label_tensor = concatenate_dict_items(team_data_tensors)

### Defining the Models

In [ ]:
epochs = 10
batch_size = 32
learning_rate = 3e-4

#Function defining the model architecture of our LSTM model
def build_LSTM_model(units = 128, n_timesteps = n_timesteps, n_features = n_features, activation = 'relu'):

    # Using LSTM model architecture
    model = Sequential([
    LSTM(units, input_shape=(n_timesteps, n_features), return_sequences=True, kernel_regularizer=l2(0.01)),
    Dropout(0.25),
    BatchNormalization(),
    LSTM(units, activation = activation, return_sequences=True, kernel_regularizer=l2(0.01)),
    Dropout(0.25),
    BatchNormalization(),
    LSTM(units, activation = activation, kernel_regularizer=l2(0.01)),
    Dropout(0.25),
    BatchNormalization(),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])
    opt = Adam(learning_rate)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model


lstm_model = build_LSTM_model()

def build_GRU_model(units = 128, n_timesteps = n_timesteps, n_features = n_features, activation = 'relu'):

    # Using GRU model architecture
    model = Sequential([
    GRU(units, input_shape=(n_timesteps, n_features), return_sequences=True, kernel_regularizer=l2(0.009)),
    Dropout(0.2),
    BatchNormalization(),
    GRU(units, activation = activation, return_sequences=True, kernel_regularizer=l2(0.009)),
    Dropout(0.2),
    BatchNormalization(),
    GRU(units, activation = activation, kernel_regularizer=l2(0.009)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
],)
    opt = Adam(learning_rate)

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


gru_model = build_GRU_model()

In [ ]:
def plot_loss_history(history, model = 'LSTM'):
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['loss'], label=f'{model} Training Loss')
    plt.plot(history.history['val_loss'], label=f'{model} Validation Loss')
    plt.title(f'Model Loss Progression for the {model} Model')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(loc='upper right')
    plt.show()

def plot_acc_history(history, model = 'LSTM'):
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['accuracy'], label=f'{model} Training Accuracy')
    plt.plot(history.history['val_accuracy'], label=f'{model} Validation Accuracy')
    plt.title(f'Model History Progression for the {model} Model')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(loc='upper right')
    plt.show()

def plot_history(history, model):
    plot_loss_history(history, model)
    plot_acc_history(history, model)


In [ ]:
history = gru_model.fit(fin_tensor, fin_label_tensor, epochs=epochs, batch_size=batch_size, validation_split=0.1)
plot_history(history, 'GRU')
history = lstm_model.fit(fin_tensor, fin_label_tensor, epochs=epochs, batch_size=batch_size, validation_split=0.1)
plot_history(history, 'LSTM')

In [ ]:
gru_model.evaluate(fin_test_tensor, fin_test_label_tensor)

In [ ]:
lstm_model.evaluate(fin_test_tensor, fin_test_label_tensor)

# Expected Profit Calculation

In [ ]:
profit_test = X_val.copy()
profit_test['predictions'] = rf_val_preds #Using random forest
profit_test['outcome'] = df['outcome']
profit_test.head()

In [ ]:
#dont bet on games with high moneylines
def calculate_profits_with_outcome(df, bet_amount=100, ml_cutoff = -250):
    total_profit = 0
    count_bets = 0
    predicted_outcomes = df.predictions
    outcomes = df.outcome
    home_moneylines = df.moneyLine
    opponent_moneylines = df.opponentMoneyLine

    for pred, outcome, home_ml, opp_ml in zip(predicted_outcomes, outcomes, home_moneylines, opponent_moneylines):
        if (pred == 0 and opp_ml >= ml_cutoff) or (pred == 1 and home_ml >= ml_cutoff):
            if pred == outcome:  # Prediction matches outcome
                if pred == 0:  # Opponent wins
                    if opp_ml > 0:
                        profit = bet_amount * (opp_ml / 100)
                    else:
                        profit = bet_amount / (-opp_ml / 100)
                    total_profit += profit
                else:  # Home team wins
                    if home_ml > 0:
                        profit = bet_amount * (home_ml / 100)
                    else:
                        profit = bet_amount / (-home_ml / 100)
                    total_profit += profit
            else:  # Prediction does not match outcome
                total_profit -= bet_amount

    return total_profit

total_profit = calculate_profits_with_outcome(profit_test, 100, -150)
total_profit

In [ ]:
profits = []
possible_ml_thresholds = np.arange(-100, -450, -25)

for i in possible_ml_thresholds:
    curr_profit = calculate_profits_with_outcome(profit_test, 100, i)
    profits.append(curr_profit)

plt.plot(possible_ml_thresholds, profits)
plt.xlabel("ML Cutoff")
plt.ylabel('Profit');

In [ ]:
test_probs = lr_model.predict_proba(X_val)
test_probs

profits = []
possible_thresholds = np.arange(0.5, 1, 0.05)

for i in possible_thresholds:
    threshold_index = np.where(test_probs > i)[0]
    curr_data = profit_test.iloc[threshold_index]
    curr_profit = calculate_profits_with_outcome(curr_data, 100, -150)
    profits.append(curr_profit)

plt.plot(possible_thresholds, profits)
plt.title('Profits From Predictions on Different Thresholds')
plt.xlabel('Confidence Threshold')
plt.ylabel('Profits')

In [ ]:
def upsets(predicted_outcomes, outcomes, home_moneylines, opponent_moneylines, bet_amount=100):
    total_profit = 0

    for pred, outcome, home_ml, opp_ml in zip(predicted_outcomes, outcomes, home_moneylines, opponent_moneylines):
        if (pred == 0 and opp_ml >= 0) or (pred == 1 and home_ml >= 0):
            if pred == outcome:  # Prediction matches outcome
                if pred == 0:  # Opponent wins
                    if opp_ml > 0:
                        profit = bet_amount * (opp_ml / 100)
                    else:
                        profit = bet_amount / (-opp_ml / 100)
                    total_profit += profit
                else:  # Home team wins
                    if home_ml > 0:
                        profit = bet_amount * (home_ml / 100)
                    else:
                        profit = bet_amount / (-home_ml / 100)
                    total_profit += profit
            else:  # Prediction does not match outcome
                total_profit -= bet_amount

    return total_profit
total_profit = upsets(profit_test.predictions,profit_test.outcome, profit_test.moneyLine, profit_test.opponentMoneyLine, 100)
total_profit